# Explore here

In [47]:
import pandas as pd
import numpy as np
import regex as re

import matplotlib.pyplot as plt
import seaborn as sns

from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.svm import SVC

import warnings
warnings.filterwarnings("ignore")

In [48]:
archivo = 'https://breathecode.herokuapp.com/asset/internal-link?id=932&path=url_spam.csv'
total_data  = pd.read_csv(archivo)
total_data.head(10)

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True
5,https://www.brookings.edu/interactives/reopeni...,False
6,https://www.reuters.com/investigates/special-r...,False
7,https://www.theatlantic.com/magazine/archive/2...,False
8,https://www.vox.com/2020/6/17/21294680/john-bo...,False
9,https://www.theguardian.com/travel/2020/jun/18...,False


### Preprocesamiento de datos

In [50]:
total_data["is_spam"] = total_data["is_spam"].apply(lambda x: 1 if x == True else 0).astype(int)

total_data.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,1
1,https://www.hvper.com/,1
2,https://briefingday.com/m/v4n3i4f3,1
3,https://briefingday.com/n/20200618/m#commentform,0
4,https://briefingday.com/fan,1


In [51]:
total_data.shape

(2999, 2)

In [52]:
# Ver duplicados
print("Análisis de duplicados:")

duplicados_exactos = total_data.duplicated().sum()
print(f"Duplicados exactos: {duplicados_exactos}")

# Ver solo duplicados en la columna de URL
duplicados_urls = total_data.duplicated(subset=['url']).sum()
print(f"URLs duplicadas: {duplicados_urls}")

# Ver si hay URLs iguales con diferentes etiquetas de Spam
total_data['url_lower'] = total_data['url'].str.lower()
conflictos = total_data.groupby('url_lower')['is_spam'].nunique()
conflictos = conflictos[conflictos > 1]
print(f"URLs con etiquetas contradictorias: {len(conflictos)}")

# Encuentra qué URLs tienen más duplicados
conteo_urls = total_data['url_lower'].value_counts()
urls_repetidas = conteo_urls[conteo_urls > 1]

print(f"URLs que aparecen más de una vez: {len(urls_repetidas)}")
print("\nTop 10 URLs más duplicadas:")
for url, count in urls_repetidas.head(10).items():
    # Muestra las etiquetas de cada URL duplicada
    etiquetas = total_data[total_data['url_lower'] == url]['is_spam'].unique()
    print(f"URL: {url[:50]}... | Veces: {count} | Etiquetas: {etiquetas}")

Análisis de duplicados:
Duplicados exactos: 630
URLs duplicadas: 630
URLs con etiquetas contradictorias: 0
URLs que aparecen más de una vez: 246

Top 10 URLs más duplicadas:
URL: https://www.bloomberg.com/tosv2.html... | Veces: 26 | Etiquetas: [1]
URL: https://www.hvper.com/... | Veces: 13 | Etiquetas: [1]
URL: https://briefingday.com/m/v4n3i4f3... | Veces: 13 | Etiquetas: [1]
URL: https://briefingday.com/fan... | Veces: 13 | Etiquetas: [1]
URL: https://briefingday.us8.list-manage.com/unsubscrib... | Veces: 13 | Etiquetas: [1]
URL: https://docs.google.com/forms/d/e/1faipqlscc99wwsu... | Veces: 11 | Etiquetas: [1]
URL: https://www.morningbrew.com/daily/r/... | Veces: 11 | Etiquetas: [1]
URL: https://www.morningbrew.com/daily/r... | Veces: 11 | Etiquetas: [1]
URL: https://www.morningbrew.com/daily/refer-a-friend... | Veces: 11 | Etiquetas: [1]
URL: https://www.morningbrew.com/... | Veces: 11 | Etiquetas: [1]


Dado que parece no haber ruido en el etiquetado de los url duplicados, se decide eliminarlos del dataset manteniendo únicamente la primera aparición.

In [53]:
# Eliminar duplicados
df_clean = total_data.drop_duplicates(keep='first')
total_data = total_data.drop('url_lower', axis=1)
df_clean = df_clean.drop('url_lower', axis=1)

print("✅ Duplicados eliminados manteniendo la primera aparición")
print(f"Dataset original: {total_data.shape}")
print(f"Dataset limpio: {df_clean.shape}")
print(f"Registros eliminados: {len(total_data) - len(df_clean)}")

✅ Duplicados eliminados manteniendo la primera aparición
Dataset original: (2999, 2)
Dataset limpio: (2369, 2)
Registros eliminados: 630


In [54]:
# Ver distribución de los datos
antes = total_data['is_spam'].value_counts(normalize=True)
despues = df_clean['is_spam'].value_counts(normalize=True)

print("Distribución ANTES:", antes)
print("Distribución DESPUÉS:", despues)

Distribución ANTES: is_spam
0    0.767923
1    0.232077
Name: proportion, dtype: float64
Distribución DESPUÉS: is_spam
0    0.897003
1    0.102997
Name: proportion, dtype: float64
